In [1]:
import pandas as pd

In [2]:
# my index column or the first column is unnamed
df = pd.read_csv("./data/progressive_moves_500.csv", index_col = 0)
df.head()

,opening_name,prev_moves,last_move
0,Slav Defense: Exchange Variation,NaN,d4
1,Slav Defense: Exchange Variation,d4,d5
2,Slav Defense: Exchange Variation,d4 d5,c4
3,Slav Defense: Exchange Variation,d4 d5 c4,c6
4,Slav Defense: Exchange Variation,d4 d5 c4 c6,cxd5


# Loading API keys

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
openai_api_key = os.getenv("openai_api_key")

## Setting up openai api call environment

In [5]:
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)

In [6]:
df.iloc[28]

opening_name               Nimzowitsch Defense: Kennedy Variation
prev_moves      d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...
last_move                                                    Bb4+
Name: 28, dtype: object

In [7]:
prev_moves, last_move = df.iloc[28]["prev_moves"], df.iloc[28]["last_move"]
prev_moves, last_move

('d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6 Qe5+ Nxe5 c4', 'Bb4+')

In [8]:
system_content_prim = '''You will be provided with a list of chess moves in Algebraic Notation,
            and your task is to analyse all previous moves and explain why last move was taken.'''

In [9]:
recursion_psuedo_code = '''
function explainMoves(moves):
    # base case
    if len(moves) == 1:
        return generateExplanation(moves[0])
    
    # recursion
    previous_explanation = explainMoves(moves[:-1])
    
    # generate explanation using returned explanation and last move in the list
    new_explanation = generateNewExplanation(previous_explanation, moves[-1])
    
    return new_explanation

function generateExplanation(move):
    return explain_the_move-{move}

function generateNewExplanation(previous_explanation, move):
    return explain_the_move_{move}_based_on_previous_explanation

'''

In [11]:
system_content_recur = f'''Assume you are a chess master.
            You will be provided with a list of chess moves in Algebraic Notation,
            and your task is to recursively analyse each previous move.
            A pseudo code for such recursive function is provided here - {recursion_psuedo_code}'''

In [12]:
user_content = f"previous moves are {prev_moves}, analyse each move and explain why the last move {last_move} was taken."

- wrong explanation dxe5: This move is a capture of the f6 pawn, gaining material advantage.
- from explanation 9, everything is wrong cause it escaped the repeat fxe5


In [15]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    # messages=[{"role": "user", "content": setup}],
    messages = [
    {
      "role": "system",
      "content": system_content_recur
    },
    {
      "role": "user",
      "content": user_content
    }
  ],
    stream=True,
)
print()
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


Here is the recursive implementation in pseudo code to analyse each move:

```
function explainMoves(moves):
    # base case
    if len(moves) == 1:
        return generateExplanation(moves[0])
    
    # recursion
    previous_explanation = explainMoves(moves[:-1])
    
    # generate explanation using returned explanation and last move in the list
    new_explanation = generateNewExplanation(previous_explanation, moves[-1])
    
    return new_explanation

function generateExplanation(move):
    return explain_the_move_{move}

function generateNewExplanation(previous_explanation, move):
    return explain_the_move_{move}_based_on_previous_explanation
```

To breakdown each move:

1. d4: This move is the Queen's Pawn Opening. It controls the central squares and prepares for development.
2. Nc6: This move is the Knight development and prepares for central control. It also prevents White from playing e4.
3. e4: This move is the King's Pawn Opening, controlling the center and preparing 